In [ ]:
pip install jyquickhelper

In [ ]:
pwd

'/content'

In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

# Sommaire

## Description du projet

### Quel sont les objectifs



L'objectif initial est de se famliariser avec le monde de la finance et les données financières. Pour cela on se concentre sur les actions boursieres Facebook et Apple, sur lesquelles on essaye de prédire le prix journalier de fermeture, la différence de prix pendant deux jours (le delta) et enfin de prendre des décisions en classifiant ................

### Quelle bases de données

Nous avons commencé par scrapper à la main le prix des actions, ce qui n'était pas efficace. Puis on a utilisé différentes API Alphavantage, Tiingo, YahooFinance.............

### Méthodologie et modèles

On utilise des méthodes de Machine Learning, et des méthodes de Deep Learning, on compare les résultat pour ne retenir que les plus performants parmi : 

1.   Regression Linéaire
2.   Random forest
3.   ...




Les données étant des series temporelles ......

### Résultats

## Code

### Récupération des données

#### Scrapping

# Selection de variables

In [ ]:
import pandas_datareader as web
import pandas as pd
start = pd.to_datetime('2012-01-01')
end = pd.to_datetime('today')

df_web_aapl = web.DataReader('AAPL', data_source='yahoo', start=start, end=end)[:2156]
df_web_aapl.columns = ['High_APPL', 'Low_APPL', 'Open_APPL', 'Close_APPL', 'Volume_APPL', 'Adj_Close_APPL']
df_web_fb = web.DataReader('FB', data_source='yahoo', start=start, end=end)[:2156]
df_web_fb.columns = ['High_FB', 'Low_FB', 'Open_FB', 'Close_FB', 'Volume_FB', 'Adj_Close_FB']
df_web_amzn = web.DataReader('AMZN', data_source='yahoo', start=start, end=end)[:2156]
df_web_amzn.columns = ['High_AMZN', 'Low_AMZN', 'Open_AMZN', 'Close_AMZN', 'Volume_AMZN', 'Adj_Close_AMZN']
df_web_googl = web.DataReader('GOOGL', data_source='yahoo', start=start, end=end)[:2156]
df_web_googl.columns = ['High_GOOGL', 'Low_GOOGL', 'Open_GOOGL', 'Close_GOOGL', 'Volume_GOOGL', 'Adj_Close_GOOGL']
df_web_vix = web.DataReader('^VIX', data_source='yahoo', start=start, end=end)[:2156]
df_web_vix.columns = ['High_VIX', 'Low_VIX', 'Open_VIX', 'Close_VIX', 'Volume_VIX', 'Adj_Close_VIX']
df_web_vix = df_web_vix[['High_VIX', 'Low_VIX', 'Open_VIX', 'Close_VIX', 'Adj_Close_VIX']]

In [ ]:
list_stocks = ["FB", "AAPL", "AMZN", "GOOGL"]

df_gafa_and_vix =pd.concat([df_web_aapl,df_web_amzn,df_web_googl,df_web_vix], axis=1)  # je mets pas celui de fb psq ca marche pas
df_gafa_and_vix.head()


,High_APPL,Low_APPL,Open_APPL,Close_APPL,Volume_APPL,Adj_Close_APPL,High_AMZN,Low_AMZN,Open_AMZN,Close_AMZN,Volume_AMZN,Adj_Close_AMZN,High_GOOGL,Low_GOOGL,Open_GOOGL,Close_GOOGL,Volume_GOOGL,Adj_Close_GOOGL,High_VIX,Low_VIX,Open_VIX,Close_VIX,Adj_Close_VIX
Date,,,,,,,,,,,,,,,,,,,,,,,
2012-01-03,14.732142,14.607142,14.621428,14.686786,302220800.0,12.566676,179.479996,175.550003,175.889999,179.029999,5110800,179.029999,334.409424,326.511505,326.796783,333.038025,7345600.0,333.038025,23.100000,22.540001,22.950001,22.969999,22.969999
2012-01-04,14.810000,14.617143,14.642858,14.765715,260022000.0,12.634213,180.500000,176.070007,179.210007,177.509995,4205200,177.509995,335.460449,330.640656,332.847839,334.474487,5722200.0,334.474487,23.730000,22.219999,23.440001,22.219999,22.219999
2012-01-05,14.948215,14.738214,14.819643,14.929643,271269600.0,12.774481,178.250000,174.050003,175.940002,177.610001,3809100,177.610001,332.317322,328.443451,331.396393,329.834839,6559200.0,329.834839,23.090000,21.340000,22.750000,21.480000,21.480000
2012-01-06,15.098214,14.972143,14.991786,15.085714,318292800.0,12.908023,184.649994,177.500000,178.070007,182.610001,7008400,182.610001,330.330322,325.220215,329.904907,325.335327,5380400.0,325.335327,21.719999,20.580000,21.240000,20.629999,20.629999
2012-01-09,15.276786,15.048214,15.196428,15.061786,394024400.0,12.887549,184.369995,177.000000,182.759995,178.559998,5056900,178.559998,323.823822,310.925934,323.573578,311.541534,11633500.0,311.541534,21.780001,21.000000,21.670000,21.070000,21.070000


In [ ]:
import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#vecteur cible
XTrain = df_gafa_and_vix[['High_APPL', 'Low_APPL', 'Open_APPL', 'Volume_APPL', 'Adj_Close_APPL','High_AMZN', 'Low_AMZN', 'Open_AMZN', 'Close_AMZN', 'Volume_AMZN', 'Adj_Close_AMZN','High_GOOGL', 'Low_GOOGL', 'Open_GOOGL', 'Close_GOOGL', 'Volume_GOOGL', 'Adj_Close_GOOGL','High_VIX', 'Low_VIX', 'Open_VIX', 'Close_VIX', 'Adj_Close_VIX' ]]
yTrain = df_gafa_and_vix[['Close_APPL']]
#lancer la régression
reg = sm.OLS(yTrain,XTrain)
resReg = reg.fit()

In [ ]:
#affichage
print(resReg.summary())

                                 OLS Regression Results                                
Dep. Variable:             Close_APPL   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          6.513e+06
Date:                Sat, 12 Dec 2020   Prob (F-statistic):                        0.00
Time:                        18:16:58   Log-Likelihood:                          877.62
No. Observations:                2156   AIC:                                     -1717.
Df Residuals:                    2137   BIC:                                     -1609.
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

Enlevons les variables de faible variance (i.e les variables quasi constantes)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selction: par test univarié la ANOVA F-value méthode est appropriée pour les données flottants 

In [ ]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

In [ ]:
import numpy as np
yT1=yTrain.values
np.ravel(yT1)
XT1=XTrain.values

test = SelectKBest(score_func=f_classif, k=4)
fit = test.fit(XT1, np.ravel(yT1))
print(fit.scores_)


[2.57019889e+03 5.11725854e+03 1.61600797e+03 3.79893573e+00
 3.72209636e+03 2.03211860e+01 2.03619892e+01 1.99857404e+01
 2.04235796e+01 6.30362740e-01 2.04235796e+01 2.14206332e+01
 2.14361385e+01 2.14807916e+01 2.12452343e+01 2.41069748e+00
 2.12452343e+01 2.14825034e+00 2.07373182e+00 2.05544982e+00
 2.27949896e+00 2.27949896e+00]


In [ ]:
features = fit.transform(XT1)
features.shape



(2156, 4)

In [ ]:
# selection de variables avec RFE

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
XT2=XTrain.values
yT1=yTrain.values
np.ravel(yT1)
model = LogisticRegression(solver='lbfgs')
rfe = RFE(model, 5)
fit = rfe.fit(XT2=1, np.ravel(yT1))

ValueError: ignored

In [ ]:
#XTrain et yTrain pour le format df
from sklearn.linear_model import LinearRegression
model = LinearRegression()#Initializing RFE model
rfe = RFE(model, 5)#Transforming data using RFE
X_rfe = rfe.fit_transform(XTrain,yTrain)  #Fitting the data to model
model.fit(X_rfe,yTrain)
print(rfe.support_)
print(rfe.ranking_)
print(XTrain.shape)

[ True  True  True False  True False False False False False False False
 False False False False False False False False False  True]
[ 1  1  1 18  1 10 14 15 13 16 11  8  5  9  6 17  7  4 12  2  3  1]
(2156, 22)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#nombre de variables 
from sklearn.model_selection import train_test_split
nof_list=np.arange(1,13)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(XT1,yT1)
    X_test_rfe = rfe.transform(X_test #reduire taille XT1 et construire Xtest)
    model.fit(X_train_rfe,yT1)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if score>high_score:
        high_score = score
        nof = nof_list[n]

print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

SyntaxError: ignored

In [ ]:
from sklearn.decomposition import PCA
# load data
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pd.read_csv(url, names=names)
array = dataframe.values


#selection
pca = PCA(n_components=3)
fit = pca.fit(XT1)
# summarize components
print("Explained Variance: %s" % fit.explained_variance_ratio_)
print(fit.components_)

Explained Variance: [9.99774747e-01 1.59137278e-04 6.61153284e-05]
[[-4.53831936e-08 -4.52350318e-08 -4.51815896e-08  9.99980282e-01
  -4.70776745e-08 -1.96074727e-06 -1.92982962e-06 -1.94515443e-06
  -1.94760134e-06  7.33831617e-04 -1.94760134e-06 -1.16028637e-06
  -1.14488827e-06 -1.15145924e-06 -1.15409173e-06  6.23682528e-03
  -1.15409173e-06  2.78633379e-09  2.70336656e-09  2.54936408e-09
   2.97736948e-09  2.97736948e-09]
 [ 1.04621989e-06  9.55041138e-07  1.00550158e-06 -2.08708004e-03
   1.03256673e-06  5.51544689e-05  5.01225421e-05  5.29413277e-05
   5.25804792e-05  9.75531534e-01  5.25804792e-05  2.27531111e-05
   2.02571536e-05  2.17284589e-05  2.13806184e-05  2.19849616e-01
   2.13806184e-05  1.13425641e-06  8.13626280e-07  9.25267505e-07
   9.72438238e-07  9.72438238e-07]
 [-1.86995819e-06 -1.80404160e-06 -1.83522925e-06 -5.92288808e-03
  -1.88803654e-06 -7.28882794e-05 -7.07937890e-05 -7.18457052e-05
  -7.19378076e-05 -2.19858273e-01 -7.19378076e-05 -4.24777065e-05
  -4.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

# feature extraction
model = ExtraTreesClassifier(n_estimators=10)
model.fit(XT1, yT1)
print(model.feature_importances_)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


ValueError: ignored

### Statistiques descriptives et nettoyage des données

### Choix des variables

### Preprocessing

### Découpage des données

### Definition des modèles

#### Phase d'apprentissage

#### Prédiction

#### Test